## Run the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius.

In [9]:
import os
from pathlib import Path
import subprocess
from PyStemmusScope import StemmusScope
from PyStemmusScope import save

#### Path to STEMMUS_SCOPE

STEMMUS_SCOPE source codes are located in the **private** repository on GitHub https://github.com/EcoExtreML/STEMMUS_SCOPE. You need to clone the repository locally and specify the path to it in the cell below. Make sure you have right access to the repository and you created a local branch for development purposes. 

In [2]:
# path to model repository
path_to_model = Path("path_to_STEMMUS_SCOPE_repository")

#### set git branch and see status

In [ ]:
dev_branch = "python-compliant"
!git -C {path_to_model} checkout {dev_branch}
!git -C {path_to_model} status

#### Update/set config files

In [4]:
# the user must provide the correct path
# for example:
path_to_config_file = path_to_model / "config_file_snellius.txt"
path_to_exe_file = path_to_model / "exe" / "STEMMUS_SCOPE"

# create an an instance of the model
model = StemmusScope(config_file=path_to_config_file, exe_file=path_to_exe_file)

# inspect model config template
model.config

{'WorkDir': '/scratch-shared/ecoextreml/stemmus_scope/',
 'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'directional': '/projects/0/einf2480/model_parameters/vegetation_property/directional/',
 'fluspect_parameters': '/projects/0/einf2480/model_parameters/vegetation_property/fluspect_parameters/',
 'leafangles': '/projects/0/einf2480/model_parameters/vegetation_property/leafangles/',
 'radiationdata': '/projects/0/einf2480/model_parameters/vegetation_property/radiationdata/',
 'soil_spectrum': '/projects/0/einf2480/model_parameters/vegetation_property/soil_spectrum/',
 'input_data': '/projects/0/einf2480/model_parameters/vegetation_property/input_data.xlsx',
 'InitialConditionPath': '/projects/0/einf2480/soil_initialcondition/',
 'NumberOfTimeSteps': '5',
 'InputPath': '',
 'OutputPath': ''}

In [5]:
# setup the model
config_path = model.setup(
    ForcingFileName="ZA-Kru_2000-2002_FLUXNET2015_Met.nc",
    NumberOfTimeSteps="5",
)
print(config_path)

/gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/ZA-Kru_2022-08-08-1446/ZA-Kru_2022-08-08-1446_config.txt


In [6]:
# inspect model config again
model.config

{'WorkDir': '/scratch-shared/ecoextreml/stemmus_scope/',
 'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'ZA-Kru_2000-2002_FLUXNET2015_Met.nc',
 'directional': '/projects/0/einf2480/model_parameters/vegetation_property/directional/',
 'fluspect_parameters': '/projects/0/einf2480/model_parameters/vegetation_property/fluspect_parameters/',
 'leafangles': '/projects/0/einf2480/model_parameters/vegetation_property/leafangles/',
 'radiationdata': '/projects/0/einf2480/model_parameters/vegetation_property/radiationdata/',
 'soil_spectrum': '/projects/0/einf2480/model_parameters/vegetation_property/soil_spectrum/',
 'input_data': '/projects/0/einf2480/model_parameters/vegetation_property/input_data.xlsx',
 'InitialConditionPath': '/projects/0/einf2480/soil_initialcondition/',
 'NumberOfTimeSteps': '5',
 'InputPath': '/gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/ZA-Kru_2022-0

#### Run the model

`model.run()` uses executable file. Here, for development purposes, we call MATLAB on the main matlab script i.e. `STEMMUS_SCOPE_exe` instead of calling `model.run()`.

In [7]:
# generate a src folder
path_to_code = path_to_model / "src"

# set matlab arguments
path_to_config = f"'{config_path}'"
command_line = f'matlab -r "STEMMUS_SCOPE_exe({path_to_config});exit;"'
args = [command_line, "-nodisplay", "-nosplash", "-nodesktop"]

# run the model
result = subprocess.run(args, cwd = path_to_code, shell=True)
result.check_returncode()

MATLAB is selecting SOFTWARE OPENGL rendering.
Opening log file:  /gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/ZA-Kru_2022-08-08-1446//java.log.32841

                            < M A T L A B (R) >
                  Copyright 1984-2021 The MathWorks, Inc.
             R2021a Update 3 (9.10.0.1684407) 64-bit (glnxa64)
                                May 27, 2021

 
To get started, type doc.
For product information, visit www.mathworks.com.
 
Reading config from /gpfs/scratch1/shared/ecoextreml/stemmus_scope/input/ZA-Kru_2022-08-08-1446/ZA-Kru_2022-08-08-1446_config.txt
The calculations start now
The calculations end now


#### save results in a netcdf file

In [10]:
required_netcdf_variables = path_to_model / "utils" / "csv_to_nc" / "required_netcdf_variables.csv"
nc_file_name = save.to_netcdf(config_path, required_netcdf_variables)
print(nc_file_name)

/gpfs/scratch1/shared/ecoextreml/stemmus_scope/output/ZA-Kru_2022-08-08-1446/ZA-Kru_2022-08-08-1446_STEMMUS_SCOPE.nc


#### Generate execuable file

In [11]:
os.environ['MATLAB_LOG_DIR'] = "."
command_line = 'mcc -m ./src/STEMMUS_SCOPE_exe.m -a ./src -d ./exe -o STEMMUS_SCOPE -R nodisplay -R singleCompThread'
result = subprocess.run(command_line, cwd = path_to_model, shell=True)
result.check_returncode()

Opening log file:  ./java.log.52377


#### check changes by git

In [12]:
!git -C {path_to_model} status

On branch python-compliant
Your branch is up to date with 'origin/python-compliant'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   exe/STEMMUS_SCOPE

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	exe/mccExcludedFiles.log
	exe/readme.txt
	exe/requiredMCRProducts.txt
	exe/run_STEMMUS_SCOPE.sh
	exe/unresolvedSymbols.txt
	java.log.52377
	test_config.txt
	utils/csv_to_nc/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")
